In [1]:
import xmltodict 
import json
import os

pokedex_path = '../raw/XmlDump/DexEntries.json'
pokedex_data_dir = '../Version20/Pokedex/'

In [2]:
book_raw = json.loads(open(pokedex_path).read())
len(book_raw)
len(os.listdir(pokedex_data_dir))
cnt = 0
for raw in book_raw:
    flag = False
    for fname in os.listdir(pokedex_data_dir):
        json_data = json.loads(open(pokedex_data_dir+'/'+fname).read())
        n = json_data['Name']
        if n == raw['Name']:
            cnt+=1
            flag = True
            break
    if not flag: print(raw['Name'])
print(cnt)

1020


# Dex Entries Merge

The name of our data is the same as the book ids, so we can merge on that. (Going forward, the _id should remain the ID used in the book (IE, Form Zacian) even if we clean up those names in the Name field.)

This'll need to be expanded for **updating existing values** and **to produce move lists**, because that'll automate 2.1 hopefully. 

In [4]:
book_raw = json.loads(open(pokedex_path).read())
book_keyed = {}
for raw in book_raw:
    book_keyed[raw['Name']] = raw

for fname in os.listdir(pokedex_data_dir):
    # fname = 'Charizard.json'
    if '.json' not in fname: continue
    path = pokedex_data_dir+fname
    data = json.loads(open(path).read())
    new = book_keyed[data['Name']]
    if new['Name'] == 'Egg': continue
    try:
        data["DexCategory"] = new.get('Category',"")
        data['Height'] = {}
        data['Height']['Deimeters'] = int(new['AverageHeight']['@Value'])
        data['Height']['Meters'] = round(int(new['AverageHeight']['@Value'])*0.1, 2)
        data['Height']['Feet'] = round(int(new['AverageHeight']['@Value'])*0.3280839895, 1)
        data['Weight'] = {}
        data['Weight']['Hectograms'] = int(new['AverageWeight']['@Value'])
        data['Weight']['Kilograms'] = round(int(new['AverageWeight']['@Value'])*0.1, 2)
        data['Weight']['Pounds'] = round(int(new['AverageWeight']['@Value'])*0.2204622622, 1)
        data["DexDescription"] = new.get('DexDescription',"")
        data['Baby'] = new.get('IsBaby') == "true"
        data['PrimaryEggGroup'] = new['PrimaryEggGroup']
        data['SecondaryEggGroup'] = new['SecondaryEggGroup'] if type(new['SecondaryEggGroup']) == str else ""
        
        data['BookImageName'] = new['PrimaryImage']['@DisplayName']
        if new.get('ShinyImage'): data['BookShinyImageName'] = new['ShinyImage']['@DisplayName']
        
        if new['MegaEvolutions']:
            data['MegaEvolutions'] = []
            extract = new['MegaEvolutions']['MegaEvolutionEntry']
            if type(extract) == dict: extract = [extract]
            for evo in extract:
                data['MegaEvolutions'].append(
                    {"Name":evo['TargetEvolution']['@DisplayName'], "Item":evo['Item']['@DisplayName']})
    except:
        print("ERROR: "+ new['Name'])
        print(new)
        raise
        # raise err
    
    # print(json.dumps(data))
    # break
    open(path,'w').write(json.dumps(data, indent=4))

# Read dem Evolution trees
One backward entry and 0 or more forward entries

Discussion about schema: https://discord.com/channels/245675629515767809/965329923256516628/1032106033277059146

In [12]:
from typing import Any, Dict, NamedTuple, Union, Set, Tuple
evolution_tree_path = '../raw/XmlDump/EvolutionTrees.json'
#
evolution_tree = json.load(open(evolution_tree_path))
#sorry, I need my type safety... Or hints at least...
Evo_tree_entry = NamedTuple('Evo_tree_entry',
    from_mon=str,
    to_mon=str,
    kind=str,
    #todo: Parse into json after being read
    details=str
)
Evo_dict = Dict[str, NamedTuple('Evo_items',
    evo_from=Evo_tree_entry,
    evo_to=list[Evo_tree_entry],
    baby_item=str
)]
mon_to_evo_vals: Evo_dict = {}
#collect the values
for tree in evolution_tree:
    #root is defined as "Does not evolve from anything"
    baby_item = tree.get("BabyEvolutionItem")
    counter_part = tree.get("BreedCounterpart")
